# Sessão 1: Configuração e Carregamento dos Dados
<div class="markdown">
Nesta primeira etapa, vamos configurar nosso ambiente e carregar o dataset que preparamos. O foco aqui é ter a base de dados pronta para as transformações que faremos. Vamos carregar apenas a tabela game_shot_charts, pois ela contém todas as informações que precisamos para criar nossas features iniciais.
</div>

In [1]:
import pandas as pd
import os
import sqlite3
import numpy as np
from sklearn.preprocessing import normalize

In [2]:
# Configurações
DB_NAME = "../nba_shots.sqlite"

# --- Carregar os Dados ---
conn = sqlite3.connect(DB_NAME)
query = "SELECT * FROM game_shot_charts"
df = pd.read_sql_query(query, conn)
conn.close()

print("Dados shot_chartscarregados com sucesso!")
print(f"Total de arremessos no dataset: {len(df)}")
display(df.head())

Dados shot_chartscarregados com sucesso!
Total de arremessos no dataset: 655447


,id,game_id,game_event_id,player_id,team_id,period,minutes_remaining,seconds_remaining,shot_made_flag,loc_x,loc_y,shot_distance,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,season
0,1,0022400001,7,1642258,1610612737,1,11,43,0,-168,205,26,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,2024-25
1,2,0022400001,10,1630552,1610612737,1,11,38,0,-136,-1,13,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,2024-25
2,3,0022400001,21,1630552,1610612737,1,10,50,1,157,203,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,2024-25
3,4,0022400001,34,1630811,1610612737,1,9,47,0,-176,184,25,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,2024-25
4,5,0022400001,36,203991,1610612737,1,9,44,1,-25,8,2,Putback Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2024-25


# Sessão 2: Criação de Novas Features (Engenharia de Características)
<div class="markdown">
A engenharia de características é onde a mágica acontece. É o processo de usar o conhecimento do domínio (neste caso, basquete) e os insights da EDA para criar novas variáveis que não existem nos dados brutos. Um bom conjunto de features é muitas vezes mais importante do que a escolha do algoritmo do modelo. Vamos criar algumas features que capturam melhor o contexto de tempo e espaço de cada arremesso.   


</div>

In [3]:
# Feature 1: Tempo Restante Total no Jogo (em segundos)
# As colunas 'period', 'minutes_remaining' e 'seconds_remaining' são úteis,
# mas um valor contínuo que represente o tempo total restante no jogo pode
# capturar melhor os efeitos de fadiga e pressão de final de jogo. [5]

def calculate_time_remaining(row):
    period = row['period']
    minutes = row['minutes_remaining']
    seconds = row['seconds_remaining']
    
    # Tempo total em um jogo de regulamentação (48 minutos = 2880 segundos)
    total_seconds_in_game = 48 * 60
    
    # Segundos decorridos nos períodos anteriores
    # Assumindo períodos de 12 minutos (720 segundos)
    seconds_elapsed_prior_periods = (period - 1) * 12 * 60
    
    # Segundos decorridos no período atual
    seconds_elapsed_current_period = (12 * 60) - (minutes * 60 + seconds)
    
    total_seconds_elapsed = seconds_elapsed_prior_periods + seconds_elapsed_current_period
    
    # Lidando com prorrogações (períodos > 4) de forma simplificada
    if period > 4:
        # Para prorrogações, o tempo restante é pequeno. Vamos atribuir um valor negativo
        # para indicar que o tempo regulamentar acabou. Cada prorrogação tem 5 min (300s).
        overtime_seconds = (period - 5) * 300 + (300 - (minutes * 60 + seconds))
        return -overtime_seconds

    return total_seconds_in_game - total_seconds_elapsed

df['time_remaining_in_game'] = df.apply(calculate_time_remaining, axis=1)

print("Feature 'time_remaining_in_game' criada.")

Feature 'time_remaining_in_game' criada.


In [4]:
# Feature 2: Ângulo do Arremesso
# As coordenadas X e Y são ótimas, mas o ângulo do arremesso em relação à cesta
# pode ser uma feature mais direta e poderosa, distinguindo arremessos de canto
# de arremessos frontais, mesmo que à mesma distância.

# A cesta está em (0, 0) nas coordenadas loc_x e loc_y.
# Usamos np.arctan2 para calcular o ângulo em radianos e depois convertemos para graus.
# O ajuste de +90 graus orienta o ângulo para que 0 grau seja "em frente à cesta".
df['shot_angle'] = np.rad2deg(np.arctan2(df['loc_x'], df['loc_y'])) + 90

print("Feature 'shot_angle' criada.")
display(df[['loc_x', 'loc_y', 'shot_angle', 'shot_distance']].head())

Feature 'shot_angle' criada.


,loc_x,loc_y,shot_angle,shot_distance
0,-168,205,50.664963,26
1,-136,-1,-0.421285,13
2,157,203,127.718331,25
3,-176,184,46.273030,25
4,-25,8,17.744672,2


# Sessão 3: Preparação dos Dados para o Modelo (Pré-processamento)
<div class="markdown">
Com nossas novas features criadas, o próximo passo é preparar o dataset final para o treinamento. Isso envolve duas etapas principais:

Seleção de Features: Escolher quais colunas vamos de fato usar para treinar o modelo. Vamos remover identificadores e colunas que já foram transformadas.

Codificação de Variáveis Categóricas: Modelos de machine learning como o XGBoost não entendem texto (ex: 'Jump Shot'). Precisamos converter essas variáveis categóricas em um formato numérico que o modelo possa processar. Usaremos a técnica de One-Hot Encoding para isso.   


</div>

In [5]:
# 1. Seleção de Features
# Selecionamos as features que acreditamos serem mais preditivas,
# incluindo as que criamos.
features_to_use = [
    'game_id',
    'player_id',
    'team_id',
    'loc_x',
    'loc_y',
    'shot_distance',
    'action_type',
    'shot_type',
    'shot_zone_basic',
    'shot_zone_area',
    'shot_zone_range',
    'time_remaining_in_game',
    'shot_angle'
]

target_variable = 'shot_made_flag'

# Criando os dataframes X (features) e y (alvo)
X = df[features_to_use]
y = df[target_variable]

print("Features selecionadas para o modelo:")
print(X.columns.tolist())

# 2. Codificação de Variáveis Categóricas
# Identificamos as colunas que são do tipo 'object' (texto)
categorical_features = [
    'action_type',
    'shot_type',
    'shot_zone_basic',
    'shot_zone_area',
    'shot_zone_range'
]

print(f"\nVariáveis categóricas a serem codificadas: {categorical_features}")

# Aplicamos o One-Hot Encoding
# Isso cria novas colunas binárias para cada categoria em nossas features de texto.
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)

print("\nDimensões do dataset de features após o One-Hot Encoding:")
print(X_encoded.shape)

print("\nAmostra do dataset final de features:")
display(X_encoded.head())

Features selecionadas para o modelo:
['game_id', 'player_id', 'team_id', 'loc_x', 'loc_y', 'shot_distance', 'action_type', 'shot_type', 'shot_zone_basic', 'shot_zone_area', 'shot_zone_range', 'time_remaining_in_game', 'shot_angle']

Variáveis categóricas a serem codificadas: ['action_type', 'shot_type', 'shot_zone_basic', 'shot_zone_area', 'shot_zone_range']

Dimensões do dataset de features após o One-Hot Encoding:
(655447, 71)

Amostra do dataset final de features:


,game_id,player_id,team_id,loc_x,loc_y,shot_distance,time_remaining_in_game,shot_angle,action_type_Alley Oop Layup shot,action_type_Cutting Dunk Shot,...,shot_zone_basic_Right Corner 3,shot_zone_area_Center(C),shot_zone_area_Left Side Center(LC),shot_zone_area_Left Side(L),shot_zone_area_Right Side Center(RC),shot_zone_area_Right Side(R),shot_zone_range_24+ ft.,shot_zone_range_8-16 ft.,shot_zone_range_Back Court Shot,shot_zone_range_Less Than 8 ft.
0,0022400001,1642258,1610612737,-168,205,26,2863,50.664963,False,False,...,False,False,True,False,False,False,True,False,False,False
1,0022400001,1630552,1610612737,-136,-1,13,2858,-0.421285,False,False,...,False,False,False,True,False,False,False,True,False,False
2,0022400001,1630552,1610612737,157,203,25,2810,127.718331,False,False,...,False,False,False,False,True,False,True,False,False,False
3,0022400001,1630811,1610612737,-176,184,25,2747,46.273030,False,False,...,False,False,True,False,False,False,True,False,False,False
4,0022400001,203991,1610612737,-25,8,2,2744,17.744672,False,False,...,False,True,False,False,False,False,False,False,False,True


In [6]:
df = X
df['shot_made_flag'] = y

In [7]:
df

,game_id,player_id,team_id,loc_x,loc_y,shot_distance,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,time_remaining_in_game,shot_angle,shot_made_flag
0,0022400001,1642258,1610612737,-168,205,26,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,2863,50.664963,0
1,0022400001,1630552,1610612737,-136,-1,13,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,2858,-0.421285,0
2,0022400001,1630552,1610612737,157,203,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,2810,127.718331,1
3,0022400001,1630811,1610612737,-176,184,25,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,2747,46.273030,0
4,0022400001,203991,1610612737,-25,8,2,Putback Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2744,17.744672,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655442,0022201218,1630550,1610612766,61,55,8,Pullup Jump shot,2PT Field Goal,In The Paint (Non-RA),Right Side(R),8-16 ft.,216,137.960936,0
655443,0022201218,1631109,1610612766,5,12,1,Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,186,112.619865,1
655444,0022201218,1630177,1610612766,-100,185,21,Pullup Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,158,61.606981,0
655445,0022201218,1630177,1610612766,1,6,0,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,121,99.462322,1


# Normalizar dados numericos

In [8]:

dados_normalizar = [
    'loc_x',
    'loc_y',
    'shot_distance',
    'time_remaining_in_game',
    'shot_angle'
]

for coluna in dados_normalizar:
    dados = X_encoded[coluna]
    dados_normalizados = normalize([dados], norm='max')[0]
    X_encoded[coluna] = dados_normalizados

X_encoded

,game_id,player_id,team_id,loc_x,loc_y,shot_distance,time_remaining_in_game,shot_angle,action_type_Alley Oop Layup shot,action_type_Cutting Dunk Shot,...,shot_zone_basic_Right Corner 3,shot_zone_area_Center(C),shot_zone_area_Left Side Center(LC),shot_zone_area_Left Side(L),shot_zone_area_Right Side Center(RC),shot_zone_area_Right Side(R),shot_zone_range_24+ ft.,shot_zone_range_8-16 ft.,shot_zone_range_Back Court Shot,shot_zone_range_Less Than 8 ft.
0,0022400001,1642258,1610612737,-0.672,0.243468,0.305882,0.994097,0.187648,False,False,...,False,False,True,False,False,False,True,False,False,False
1,0022400001,1630552,1610612737,-0.544,-0.001188,0.152941,0.992361,-0.001560,False,False,...,False,False,False,True,False,False,False,True,False,False
2,0022400001,1630552,1610612737,0.628,0.241093,0.294118,0.975694,0.473031,False,False,...,False,False,False,False,True,False,True,False,False,False
3,0022400001,1630811,1610612737,-0.704,0.218527,0.294118,0.953819,0.171382,False,False,...,False,False,True,False,False,False,True,False,False,False
4,0022400001,203991,1610612737,-0.100,0.009501,0.023529,0.952778,0.065721,False,False,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655442,0022201218,1630550,1610612766,0.244,0.065321,0.094118,0.075000,0.510966,False,False,...,False,False,False,False,False,True,False,True,False,False
655443,0022201218,1631109,1610612766,0.020,0.014252,0.011765,0.064583,0.417111,False,False,...,False,True,False,False,False,False,False,False,False,True
655444,0022201218,1630177,1610612766,-0.400,0.219715,0.247059,0.054861,0.228174,False,False,...,False,False,True,False,False,False,False,False,False,False
655445,0022201218,1630177,1610612766,0.004,0.007126,0.000000,0.042014,0.368379,False,False,...,False,True,False,False,False,False,False,False,False,True


In [9]:
y

0         0
1         0
2         1
3         0
4         1
         ..
655442    0
655443    1
655444    0
655445    1
655446    1
Name: shot_made_flag, Length: 655447, dtype: int64

# Exportar arquivos

In [10]:
# Salvar o DataFrame como CSV
X_encoded.to_csv('../data/X_encoded.csv', index=False)
y.to_csv('../data/y.csv', index=False)
X.to_csv('../data/X.csv', index=False)
df.to_csv('../data/df.csv', index=False)